In [1]:
import os
import sys
import math
import random
import joblib
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))

# 0. 데이터 전처리(판매량 및 날짜 생성)

In [3]:
sale.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,옵션,종류,브랜드,상품명다시,단위
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,NaN,0,테이트,남성 셀린니트,3종
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,0.0,NaN,0,테이트,여성 셀린니트,3종
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,0.0,NaN,0,테이트,남성 셀린니트,3종
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,0.0,NaN,0,테이트,여성 셀린니트,3종
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,0.0,NaN,0,테이트,남성 셀린니트,3종


In [4]:
# 판매량
import math
sale['판매량'] = sale['취급액'] / sale['판매단가']
sale['판매량'] = sale['판매량'].fillna(0).apply(lambda x : math.ceil(x))

In [5]:
sale['방송날'] = sale['방송일시'].dt.date
sale['방송월'] = sale['방송일시'].dt.month
sale['방송일'] = sale['방송일시'].dt.day
sale['방송시간'] = sale['방송일시'].dt.hour

In [6]:
sale_a = sale.loc[sale['방송일시'] < '2019-11-01 02:00:00']

In [7]:
sale_b = sale.loc[sale['방송일시'] > '2019-11-01 02:00:00']

In [8]:
sale_a['방송날'] = sale_a.apply(lambda x : datetime.datetime.combine(x['방송날'], datetime.time(x['방송시간'])), axis = 1)

In [9]:
temp_a = sale_a.drop_duplicates(['방송월', '방송일', '방송시간'])[['방송날', '마더코드', 'NEW상품명', 'NEW상품코드', '상품군', '판매량']]

In [10]:
sale_b['new_방송일시'] = sale_b['방송일시'] - datetime.timedelta(minutes = 20)

In [11]:
sale_b['방송날'] = sale_b['new_방송일시'].dt.date
sale_b['방송월'] = sale_b['new_방송일시'].dt.month
sale_b['방송일'] = sale_b['new_방송일시'].dt.day
sale_b['방송시간'] = sale_b['new_방송일시'].dt.hour

In [12]:
sale_b['방송날'] = sale_b.apply(lambda x : datetime.datetime.combine(x['방송날'], datetime.time(x['방송시간'])), axis = 1)

In [13]:
temp_b = sale_b.drop_duplicates(['방송월', '방송일', '방송시간'])[['방송날', '마더코드', 'NEW상품명', 'NEW상품코드', '상품군', '판매량']]

In [14]:
df = pd.concat([temp_a, temp_b], axis = 0)

In [15]:
df['방송일'] = pd.to_datetime(df['방송날']).dt.date
df['방송시간'] = pd.to_datetime(df['방송날']).dt.hour

In [16]:
df

,방송날,마더코드,NEW상품명,NEW상품코드,상품군,판매량,방송일,방송시간
0,2019-01-01 06:00:00,100346,테이트 남성 셀린니트 3종,865.0,의류,53,2019-01-01,6
6,2019-01-01 07:00:00,100305,오모떼 여성 레이스 파운데이션 브라,632.0,속옷,116,2019-01-01,7
9,2019-01-01 08:00:00,100808,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,1079.0,의류,270,2019-01-01,8
12,2019-01-01 09:00:00,100816,보코 리버시블 무스탕,378.0,의류,204,2019-01-01,9
15,2019-01-01 10:00:00,100809,CERINI by PAT 남성 풀패키지 기모니트 3종,1089.0,의류,611,2019-01-01,10
...,...,...,...,...,...,...,...,...
38283,2019-12-31 21:00:00,100155,LG 매직스페이스 냉장고,1123.0,가전,22,2019-12-31,21
38289,2019-12-31 22:00:00,100543,NaN,NaN,무형,0,2019-12-31,22
38292,2019-12-31 23:00:00,100448,쿠첸 압력밥솥 10인용,818.0,주방,26,2019-12-31,23
38304,2020-01-01 00:00:00,100073,NaN,NaN,무형,0,2020-01-01,0


In [17]:
df = df.loc[df['상품군'] != '무형']

# 상품군 x 방송날 matrix

In [134]:
df = df.sort_values(['방송시간', '방송날'])

In [152]:
pd.pivot_table(df, columns = '방송날', index = '상품군', values = 'NEW상품코드')

방송날,2019-01-01 06:00:00,2019-01-01 07:00:00,2019-01-01 08:00:00,2019-01-01 09:00:00,2019-01-01 10:00:00,2019-01-01 11:00:00,2019-01-01 12:00:00,2019-01-01 13:00:00,2019-01-01 14:00:00,2019-01-01 15:00:00,...,2019-12-31 12:00:00,2019-12-31 14:00:00,2019-12-31 15:00:00,2019-12-31 16:00:00,2019-12-31 17:00:00,2019-12-31 18:00:00,2019-12-31 19:00:00,2019-12-31 20:00:00,2019-12-31 21:00:00,2019-12-31 23:00:00
상품군,,,,,,,,,,,,,,,,,,,,,
가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,132.0,NaN,NaN,NaN
가전,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1123.0,NaN
건강기능,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
농수축,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,407.0,...,NaN,NaN,1327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
생활용품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN
속옷,NaN,632.0,NaN,NaN,NaN,831.0,NaN,629.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
의류,865.0,NaN,1079.0,378.0,1089.0,NaN,NaN,NaN,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,250.0,NaN,NaN,NaN,NaN
이미용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
잡화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
data = df.sort_values(['상품군', '방송일'])[['상품군', 'NEW상품코드']].reset_index(drop = True)
data

,상품군,NEW상품코드
0,가구,341.0
1,가구,341.0
2,가구,341.0
3,가구,341.0
4,가구,449.0
...,...,...
7289,침구,370.0
7290,침구,370.0
7291,침구,369.0
7292,침구,368.0


In [153]:
data['상품군'].value_counts()

농수축     1228
주방      1080
의류      1061
속옷       976
잡화       598
가전       573
생활용품     494
이미용      425
무형       370
건강기능     222
가구       198
침구        69
Name: 상품군, dtype: int64

# 시간 x 방송일 matrix

In [224]:
data = df[['방송일', '방송시간', 'NEW상품코드']].reset_index(drop = True)

In [225]:
pd.pivot_table(data, index = '방송시간', columns = '방송일', values = 'NEW상품코드')

방송일,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
방송시간,,,,,,,,,,,,,,,,,,,,,
0,NaN,30.0,865.0,256.0,489.0,402.0,476.0,1099.0,530.0,402.0,...,NaN,1290.0,157.0,1039.0,NaN,1223.0,NaN,NaN,65.0,652.0
1,NaN,140.0,928.0,322.0,4.0,1348.0,84.0,928.0,4.0,140.0,...,974.0,781.0,651.0,1074.0,1276.0,1258.0,1290.0,543.0,780.0,174.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,865.0,322.0,597.0,1328.0,481.0,742.0,139.0,481.0,378.0,269.0,...,304.0,543.0,250.0,79.0,609.0,842.0,687.0,292.0,637.0,838.0
7,632.0,741.0,747.0,780.0,535.0,255.0,607.0,241.0,741.0,241.0,...,1078.0,NaN,930.0,244.0,1362.0,1323.0,79.0,328.0,1323.0,1074.0
8,1079.0,632.0,607.0,167.0,241.0,162.0,378.0,812.0,139.0,530.0,...,847.0,648.0,847.0,823.0,47.0,587.0,471.0,1162.0,1362.0,1162.0
9,378.0,1154.0,154.0,236.0,644.0,378.0,402.0,607.0,235.0,629.0,...,773.0,47.0,526.0,622.0,1046.0,38.0,622.0,1202.0,526.0,69.0
10,1089.0,819.0,629.0,824.0,378.0,632.0,629.0,824.0,607.0,235.0,...,982.0,1314.0,746.0,315.0,1287.0,847.0,16.0,332.0,1287.0,1039.0


In [226]:
data['방송시간'].value_counts()

21    365
9     365
15    365
11    365
8     365
22    365
16    365
10    365
20    365
1     365
19    365
17    365
13    365
12    364
23    364
6     364
18    364
7     364
0     364
14    363
5       1
2       1
Name: 방송시간, dtype: int64

In [227]:
data = data.loc[data['NEW상품코드'].notnull()]

In [228]:
data['NEW상품코드'] = data['NEW상품코드'].astype(int)

In [229]:
data = data[(data['방송시간'] != 5) & (data['방송시간'] != 2)]

In [230]:
data[['방송시간', 'NEW상품코드']].to_csv('./nsshop_ver2.csv', encoding = 'cp949')

In [231]:
data

,방송일,방송시간,NEW상품코드
0,2019-01-02,0,30
1,2019-01-03,0,865
2,2019-01-04,0,256
3,2019-01-05,0,489
4,2019-01-06,0,402
...,...,...,...
7285,2019-12-23,23,726
7288,2019-12-26,23,979
7291,2019-12-29,23,458
7292,2019-12-30,23,920


In [213]:
data['방송월'] = pd.to_datetime(data['방송일']).dt.month

In [214]:
data['code'] = data['방송월'].astype(str) + '-' + data['방송시간'].astype(str)

In [202]:
meta = data[['code', '방송월']].drop_duplicates().reset_index(drop = True).reset_index()

In [219]:
res = data.merge(meta, on = 'code', how = 'left')[['index', 'NEW상품코드']]

In [221]:
res.to_csv('./nsshop.csv', encoding = 'cp949')

In [222]:
res

,index,NEW상품코드
0,0,30
1,0,865
2,0,256
3,0,489
4,0,402
...,...,...
6917,239,726
6918,239,979
6919,239,458
6920,239,920


# 시간 x 아이템 matrix

In [41]:
temp = pd.pivot_table(df, index = '방송시간', columns = 'NEW상품코드', values = '판매량')
temp

NEW상품코드,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1354.0,1355.0,1356.0,1357.0,1358.0,1359.0,1360.0,1361.0,1362.0,1363.0
방송시간,,,,,,,,,,,,,,,,,,,,,
0,88.500000,NaN,NaN,107.500000,NaN,69.000,NaN,58.0,NaN,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.000000,40.0,281.0,75.166667,57.25,30.875,NaN,NaN,NaN,NaN,...,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.666667,NaN,...,NaN,NaN,NaN,NaN,203.0,NaN,NaN,NaN,206.333333,308.0
7,NaN,NaN,NaN,NaN,52.00,NaN,69.0,NaN,NaN,NaN,...,NaN,NaN,36.5,NaN,NaN,NaN,NaN,NaN,277.000000,NaN
8,NaN,NaN,NaN,NaN,80.00,NaN,NaN,NaN,NaN,NaN,...,574.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227.333333,NaN
9,NaN,NaN,NaN,NaN,44.00,91.000,NaN,NaN,NaN,NaN,...,NaN,128.0,69.0,NaN,NaN,NaN,NaN,NaN,389.750000,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,80.0,NaN,782.000000,NaN,NaN,379.000000,NaN,NaN,NaN


In [51]:
(temp == 0).sum().sort_values()

NEW상품코드
0.0       0
898.0     0
899.0     0
900.0     0
902.0     0
         ..
992.0     2
254.0     3
1129.0    3
447.0     3
760.0     3
Length: 1133, dtype: int64

In [53]:
temp.loc[temp[760] == 0]

NEW상품코드,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1354.0,1355.0,1356.0,1357.0,1358.0,1359.0,1360.0,1361.0,1362.0,1363.0
방송시간,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,52.0,NaN,69.0,NaN,NaN,NaN,...,NaN,NaN,36.5,NaN,NaN,NaN,NaN,NaN,277.0,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,661.5,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,974.0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
sale.loc[(sale['NEW상품명'] == '캐리어 벽걸이에어컨 7형') & (sale['방송일'] == 13)]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,옵션,종류,브랜드,상품명다시,단위,판매량,방송날,방송월,방송일,방송시간
17464,2019-06-13 21:00:00,NaN,100067,200174,일시불 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,709000,NaN,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,0,2019-06-13,6,13,21
17467,2019-06-13 21:00:00,NaN,100067,200166,무이자 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,749000,NaN,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,0,2019-06-13,6,13,21
17470,2019-06-13 21:15:00,NaN,100067,200174,일시불 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,709000,4159000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,6,2019-06-13,6,13,21
17471,2019-06-13 21:15:00,NaN,100067,200166,무이자 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,749000,1498000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,2,2019-06-13,6,13,21
17474,2019-06-13 21:30:00,NaN,100067,200174,일시불 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,709000,1420000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,3,2019-06-13,6,13,21
17475,2019-06-13 21:30:00,NaN,100067,200166,무이자 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,749000,8778000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,12,2019-06-13,6,13,21
17476,2019-06-13 21:45:00,NaN,100067,200166,무이자 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,749000,8758000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,12,2019-06-13,6,13,21
17477,2019-06-13 21:45:00,NaN,100067,200174,일시불 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,709000,8288000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,12,2019-06-13,6,13,21
20549,2019-07-13 21:20:00,NaN,100067,200175,일시불 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,709000,3909000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,6,2019-07-13,7,13,21
20550,2019-07-13 21:20:00,NaN,100067,200167,무이자 캐리어 벽걸이에어컨 7형 ARC07VBC,가전,749000,2090000.0,760.0,캐리어 벽걸이에어컨 7형,...,NaN,에어컨,캐리어,벽걸이에어컨,7형,3,2019-07-13,7,13,21


In [58]:
df.loc[df['NEW상품코드'] == 760]

,방송날,마더코드,NEW상품명,NEW상품코드,상품군,판매량,방송일,방송시간
15449,2019-05-25 16:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,2,2019-05-25,16
16218,2019-06-01 23:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,16,2019-06-01,23
17464,2019-06-13 21:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,0,2019-06-13,21
17585,2019-06-14 22:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,2,2019-06-14,22
17618,2019-06-15 07:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,0,2019-06-15,7
17705,2019-06-15 19:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,0,2019-06-15,19
17717,2019-06-15 20:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,20,2019-06-15,20
18403,2019-06-22 23:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,14,2019-06-22,23
19806,2019-07-06 22:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,33,2019-07-06,22
21151,2019-07-18 21:00:00,100067,캐리어 벽걸이에어컨 7형,760.0,가전,0,2019-07-18,21


In [55]:
temp[760]

방송시간
0      NaN
1      NaN
2      NaN
5      NaN
6      NaN
7      0.0
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     2.0
17     NaN
18     NaN
19     0.0
20    20.0
21     0.0
22    17.5
23    15.0
Name: 760.0, dtype: float64

In [29]:
temp.notnull().sum().sum() / (temp.shape[0] * temp.shape[1])

0.14358501163443793

In [33]:
temp = pd.pivot_table(df, index = '방송시간', columns = '마더코드', values = '판매량')
temp

마더코드,100000,100001,100002,100006,100009,100010,100013,100015,100016,100017,...,100837,100838,100839,100840,100841,100843,100844,100845,100846,100849
방송시간,,,,,,,,,,,,,,,,,,,,,
0,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1562.500000,NaN,NaN,NaN,NaN,NaN
1,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,705.857143,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,499.25,...,39.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,...,46.000000,NaN,NaN,NaN,NaN,NaN,574.000000,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.000000,NaN,NaN,NaN,NaN,NaN,1022.000000,NaN,NaN,NaN
10,NaN,NaN,NaN,38.0,NaN,379.000000,NaN,NaN,NaN,NaN,...,27.714286,8.50,NaN,NaN,NaN,NaN,903.000000,13.0,NaN,NaN


In [75]:
sale['옵션'] = sale['옵션'].fillna(0)
sale['옵션'].apply(lambda x : 1 if x != 0 else 0).value_counts()

0    34169
1     4140
Name: 옵션, dtype: int64

In [40]:
sale.loc[sale['상품명'].str.contains('간고등어')].drop_duplicates(['방송월', '방송일']).head(60)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,옵션,종류,브랜드,상품명다시,단위,판매량,방송날,방송월,방송일,방송시간
110,2019-01-02 15:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,41722000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,1351,2019-01-02,1,2,15
301,2019-01-04 12:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,30671000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,993,2019-01-04,1,4,12
581,2019-01-07 16:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,25845000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,837,2019-01-07,1,7,16
775,2019-01-09 18:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,31916000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,1033,2019-01-09,1,9,18
836,2019-01-10 14:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,34544000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,1118,2019-01-10,1,10,14
1228,2019-01-14 13:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,35000000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,1133,2019-01-14,1,14,13
1331,2019-01-15 16:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,30818000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,998,2019-01-15,1,15,16
1709,2019-01-19 16:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,46308000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,1499,2019-01-19,1,19,16
1914,2019-01-21 15:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,25819000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,836,2019-01-21,1,21,15
2298,2019-01-25 13:00:00,20.0,100253,200864,안동간고등어 20팩,농수축,30900,22238000.0,1277.0,안동간고등어 20팩,...,NaN,해산물,NaN,안동간고등어,20팩,720,2019-01-25,1,25,13


In [76]:
df['방송월'] = df['방송날'].dt.month

In [80]:
df['계절'] = df['방송월'].apply(lambda x : '봄' if 3 <= x <= 5 else
                            ('여름' if 6 <= x <= 8 else
                             ('가을' if 9 <= x <= 11 else 
                              ('겨울' if x < 3 or x > 11 else x))))

In [83]:
df['user'] = df['계절'] + df['방송시간'].astype(str)

In [135]:
pivT = pd.pivot_table(df, index = 'user', columns = 'NEW상품코드', values = '판매량')

In [136]:
pivT

NEW상품코드,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1354.0,1355.0,1356.0,1357.0,1358.0,1359.0,1360.0,1361.0,1362.0,1363.0
user,,,,,,,,,,,,,,,,,,,,,
가을0,88.5,NaN,NaN,44.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
가을1,13.0,NaN,NaN,51.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
가을10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,80.0,NaN,782.0,NaN,NaN,NaN,NaN,NaN,NaN
가을11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,843.8,NaN,NaN,452.5,NaN,NaN,NaN
가을12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,597.5,NaN,653.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
여름23,NaN,NaN,NaN,60.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
여름6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,203.0,NaN,NaN,NaN,NaN,NaN
여름7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
pivT.index.name = None

In [138]:
pivT = pivT+1
pivT = pivT.fillna(0)

In [144]:
pivT = pivT.reset_index()

In [147]:
pivT.index.name

In [141]:
pivT.to_excel(os.path.join('..', '..', '0.Data', '05_분석데이터', 'Rec_user_item_matrix.xlsx'), index = False)

In [102]:
from sklearn.decomposition import PCA

In [108]:
pca = PCA(n_components = 0.95)

In [109]:
pca.fit(pivT.fillna(0))

PCA(n_components=0.95)

In [110]:
pd.DataFrame(pca.fit_transform(pivT.fillna(0)))

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-569.114651,117.770771,-161.141881,142.578205,324.726571,-443.234857,505.964097,131.671292,-705.131093,287.403993,...,-242.857685,-28.612771,-70.503642,429.174706,694.043996,50.330392,-49.516855,-160.939773,-144.472358,-148.918383
1,-425.878706,234.969883,-51.763976,-11.144648,388.135088,-513.716153,242.937063,336.908200,-492.525974,196.820560,...,-167.262595,161.051450,2.039793,209.005530,304.671440,2.878704,1.294489,-82.029566,-13.968575,-204.862064
2,389.448043,-211.978020,2469.179676,-618.812927,-663.127599,-371.054108,2417.760097,-1495.551980,1600.148465,283.903126,...,-289.364816,-115.625309,-22.992649,-37.289281,-34.303902,80.921367,-17.675244,-62.858455,21.765649,51.715175
3,1034.850124,-709.106541,1439.242356,-174.383790,-689.891289,332.310916,-57.914073,117.414340,57.574650,-373.801233,...,-238.236083,5.359520,240.266386,180.578345,62.340297,285.389013,-9.375862,425.875993,-107.346875,-267.154949
4,-225.117243,658.947808,535.196637,-110.824658,318.677080,-149.527146,-46.605911,1135.165149,280.427023,-592.910615,...,161.092735,328.021830,165.546306,206.190794,-196.481021,-136.459722,-131.709668,523.694102,74.717486,94.288553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,-1054.066169,-298.286615,-37.572642,671.178265,209.279350,296.015045,-86.714000,37.072696,-221.428505,-611.515002,...,-380.706280,354.049482,-223.431428,-96.631121,52.959384,92.845190,263.661542,-174.381144,412.261723,363.016910
78,-985.209933,-131.266402,-85.922804,603.258411,151.796713,299.455720,61.186081,48.397553,-102.912441,-178.527956,...,-115.230627,129.730291,-174.714629,-157.362578,9.077816,224.950505,-105.106237,47.849190,-115.970530,361.050745
79,-1144.692522,-208.931120,19.975982,860.742484,370.577539,600.730458,103.574971,58.679105,-120.966598,-563.226349,...,-395.506412,90.999953,258.395097,-344.788015,101.524687,24.112212,190.625992,-6.424699,170.226748,-35.418939
80,-1107.784910,-116.543091,13.649139,701.398165,215.693065,356.327775,-34.624587,-23.898422,242.117783,-346.837937,...,-272.754378,-135.223119,152.204396,373.240630,-139.953340,-319.190877,130.581725,157.068994,275.147599,145.054311
